# Semantic Kernel 

In this code sample, you will use the [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework to create a basic agent. 

The goal of this sample is to show you the steps that we will later use in the additional code samples when implementing the different agentic patterns. 

## Import the Needed Python Packages 

In [ ]:
import os #
from typing import Annotated #
from openai import AsyncOpenAI #

from dotenv import load_dotenv #

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion #
from semantic_kernel.functions import kernel_function #

## Creating the Client

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to use the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `OpenAIChatCompletion` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin: 
    '''travel_agent = DestinationPlugin()
    suggestion1 = travel_agent.get_random_destination()
    '''
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]: 
        """Imported Annotated from typing: allows you to add extra metadata(comments) to a type."""
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1: #掐头去尾
            available_destinations.remove(self.last_destination)    

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
"""6.9 16:20"""
class DestinaitonsPlugin:
    def __init__(self):
        self.destinations = [
            "canada",
            'us'
        ]

        self.last = None

    @kernel_function(name='generate_destinations', description="Randomly generate a travel destination")
    def travel_destination(self)->str:
        available_destinations = self.destinations.copy()
        self.last = current_dest
        if self.last and len(available_destinations) > 1:
            available_destinations.remove(self.last)
        
        current_dest = random.choice(available_destinations)
        """random.choice() expects a sequence (like a list or tuple) as its argument"""

        return current_dest


In [ ]:
"""A practice when learning python function part"""
class Employee:
    rate = 1.04
    def __init__(self, first, last):
        self.first = first
        self.last = last
        self.email = first + "." + last + "@company.com"
    def fullname(self):
        return self.first + ' ' + self.last
    @classmethod
    def set_rate(cls, amount):
        cls.rate = amount

emp1 = Employee('Ivana', 'Lee')
emp2 = Employee('yifei', 'liu')
Employee.set_rate(1.3)
emp1.set_rate(1.05)
print(emp2.rate)
print(emp1.rate)

In [ ]:
load_dotenv()
"""This function loads environment variables from a .env file into your Python environment."""
# from openai import AsyncOpenAI
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
# from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the Agent 

Below we create the Agent called `TravelAgent`.

For this example, we are using very simple instructions. You can change these instructions to see how the agent responds differently. 

In [44]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin(), ], # The plugins parameter expects a list of plugin instances.
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Running the Agent

Now we can run the Agent by defining a thread of type `ChatHistoryAgentThread`.  Any required system messages are provided to the agent's invoke_stream `messages` keyword argument.

After these are defined, we create a `user_inputs` that will be what the user is sending to the agent. In this case, we have set this message to `Plan me a sunny vacation`. 

Feel free to change this message to see how the agent responds differently. 

In [48]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "plan a trip for me",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

# User: plan a trip for me

# TravelAgent: I've planned a trip for you to **Paris, France**! Here are some highlights you might consider for your visit:

### Top Attractions:
1. **Eiffel Tower** - Don’t miss the stunning views from the top!
2. **Louvre Museum** - Home to thousands of works of art, including the Mona Lisa.
3. **Notre-Dame Cathedral** - A masterpiece of French Gothic architecture.
4. **Montmartre** - Explore the charming streets and visit the Basilica of Sacré-Cœur.
5. **Champs-Élysées and the Arc de Triomphe** - Perfect for a leisurely stroll and some shopping.

### Suggested Itinerary:
#### Day 1:
- Arrive in Paris
- Visit the Eiffel Tower in the evening for a sunset view

#### Day 2:
- Morning visit to the Louvre Museum
- Afternoon walk in the Tuileries Garden
- Dinner in the Latin Quarter

#### Day 3:
- Explore Montmartre, visit the Basilica of Sacré-Cœur, and enjoy the local art scene
- Evening: Seine River cruise

#### Day 4:
- Visit Notre-Dame Cathedral and Sainte